# gravity_boids

Gravity-like boids.

In [8]:
class Point2d:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    # Subtract two points to get a vector start_point --> self.
    def __sub__(self, start_point):
        return Vector2d(self.x - start_point.x, self.y - start_point.y)

    # Add this point to a vector to get a new point
    # (Or add the coordinates of two points, e.g. to calculate an average.)
    def __add__(self, other):
        return Point2d(self.x + other.x, self.y + other.y)

    # Scaling isn't really a point operation.
    # It's used by this program to calculate averages.
    # Scale this point by multiplying.
    def __mul__(self, scale):
        return Point2d(self.x * scale, self.y * scale)

    # Scale this point by dividing.
    def __truediv__(self, scale):
        return self * (1.0 / scale)

    def __str__(self):
        return f'({self.x}, {self.y})'

    def draw(self, canvas, color):
        RADIUS = 3
        canvas.create_oval(
            self.x - RADIUS, self.y - RADIUS,
            self.x + RADIUS, self.y + RADIUS,
            fill=color, outline=color)

In [9]:
import math

class Vector2d:
    # Initialize from either (a, b) coordinates or two points a --> b.
    def __init__(self, a, b):
        if type(a) is Point2d:
            # Initialize from two points a --> b.
            self.x = b.x - a.x
            self.y = b.y - a.y
        else:
            # Initialize from (a, b) coordinates.
            self.x = a
            self.y = b

    # Add two vectors to get a new vector or
    # add the vector to a point to get a new point.
    def __add__(self, other):
        if type(other) is Vector2d:
            # Add two vectors to get a new vector.
            return Vector2d(self.x + other.x, self.y + other.y)
        else:
            # Add the vector to a point to get a new point.
            return Point2d(self.x + other.x, self.y + other.y)

    # Return the negation of this vector.
    def __neg__(self):
        return Vector2d(-self.x, -self.y)

    # Subtract two vectors.
    def __sub__(self, other):
        return self + -other

    # Scale this vector by multiplying.
    def __mul__(self, scale):
        return Vector2d(self.x * scale, self.y * scale)

    # Scale this vector by dividing.
    def __truediv__(self, scale):
        return self * (1.0 / scale)

    # Scale the vector.
    # Return self so we can use it in further calculations.
    def scale(self, scale):
        self.x *= scale
        self.y *= scale
        return self

    # Return the vector's length.
    def length(self):
        return math.sqrt(self.x * self.x + self.y * self.y)

    # Set the vector's length to new_length.
    # Return self so we can use it in further calculations.
    def set_length(self, new_length):
        old_length = self.length()
        if old_length < 0.01: return self # Don't divide by zero.
        self.x *= new_length / old_length
        self.y *= new_length / old_length
        return self

    # Set the vector's length to 1.
    # Return self so we can use it in further calculations.
    def normalize(self):
        return self.set_length(1)

    def __str__(self):
        return f'<{self.x}, {self.y}>'

    def draw(self, canvas, start_point, color):
        end_point = self + start_point
        canvas.create_line(
            start_point.x, start_point.y,
            end_point.x, end_point.y,
            fill=color)

In [47]:
import math

class Boid:
    def __init__(self, canvas, radius, color, mass, location, velocity):
        # Save the parameters.
        self.canvas = canvas
        self.radius = radius
        self.mass = mass
        self.location = location
        self.velocity = velocity

        # Make the boid's circle.
        self.circle = self.canvas.create_oval(
            location.x - radius, location.y - radius,
            location.x + radius, location.y + radius,
            fill=color, outline=color)

    def __str__(self):
        return f"Boid({self.mass}, {self.location}, {self.velocity})"

    # Return a vector self --> other.
    def vector_to(self, other):
        return other.location - self.location

    # Return the distance between this boid and another one.
    def distance(self, other):
        return self.vector_to(other).length()

    def update(self, boids, elapsed_time, target, max_speed, attraction_wgt,
            repulsion_wgt, target_wgt, neighborhood_distance):
        # Start with no forces.
        attraction_vector = Vector2d(0, 0)
        repulsion_vector = Vector2d(0, 0)

        # Add contributions by other boids.
        for other in boids:
            if other == self: continue  # Don't compare to self.

            # Get the distance to the other boid.
            # Set dist to the distance between the two boids. 
            # If dist < 0.1, set dist to 0.1 (so we don’t divide by zero). 
            # If dist is greater than the neighborhood distance, skip this boid.

            dist = self.distance(other)
            if dist < 0.1: dist = 0.1
            if dist > neighborhood_distance: continue

            # Calculate the attractive force due to the other boid and add it to attraction_vector. 
            # (Hint: Get the direction of the vector by using the statement vector = self.vector_to(other).normalize(). 
            # Then use the equation Fg = m1 * m2 / d2 to multiply that vector by the necessary length.)
            vector = self.vector_to(other).normalize()
            m1 = self.mass
            m2 = other.mass

            # Attractive force.          
            fg = m1 * m2 / (dist ** 2)
            attraction_vector = vector * fg

            # Calculate the repulsive force due to the other boid and add it to repulsion_vector.
            # (Hint: Use -vector to set the new force’s direction. 
            # Then use the equation FA = m1 * m2 / d3 to set its length.)
            # Repulsive force.
            fa = m1 * m2 / (dist ** 3)
            repulsion_vector = -vector * fa 

        # Add the target's contribution.
        # Ft = m1 * mT / d
        ft = self.mass * target.mass / dist
        target_attraction_vector = self.vector_to(target).normalize() * ft 

        # print(attraction_vector, repulsion_vector, target_attraction_vector)

        # Combine the force vectors.
        # Multiply attraction_vector, repulsion_vector, and target_attraction_vector 
        # by their weight factors and add them together to get the total force_vector.
        total_force_vector = \
            attraction_vector * attraction_wgt + \
            repulsion_vector * repulsion_wgt + \
            target_attraction_vector * target_wgt 

        # Convert force into acceleration.
        # Use the equation a = F / m to calculate the acceleration_vector due to the total force.
        acceleration_vector = total_force_vector / self.mass

        # Scale acceleration by elapsed time.
        acceleration_vector *= elapsed_time

        self.velocity += acceleration_vector
        # Update the velocity.
        self.new_velocity = self.velocity
 
        # Enforce the speed limit.
        if self.new_velocity.length() > max_speed:
            self.new_velocity.set_length(max_speed)

    # Move the boid to its new location.
    def move(self, elapsed):
        self.velocity = self.new_velocity
        self.location += self.velocity * elapsed
        self.move_circle()

    # Move the boid's circle to its current position.
    def move_circle(self):
        self.canvas.moveto(self.circle,
            self.location.x - self.radius,
            self.location.y - self.radius)

In [48]:
import tkinter as tk

# Get the text in an Entry widget and
# convert it to an int.
def get_int(entry):
    return int(entry.get())

# Get the text in an Entry widget and
# convert it to a float.
def get_float(entry):
    return float(entry.get())

# Make Label and Entry widgets for a field.
# Return the Entry widget.
def make_field(parent, label_width, label_text, entry_width, entry_default):
    frame = tk.Frame(parent)
    frame.pack(side=tk.TOP)

    label = tk.Label(frame, text=label_text, width=label_width, anchor=tk.W)
    label.pack(side=tk.LEFT)

    entry = tk.Entry(frame, width=entry_width, justify='right')
    entry.insert(tk.END, entry_default)
    entry.pack(side=tk.LEFT)

    return entry

In [49]:
import tkinter as tk
from timeit import default_timer as timer
import random

# Geometry constants.
MARGIN = 5
WINDOW_WID = 500
WINDOW_HGT = 300
CANVAS_HGT = WINDOW_HGT - 2 * MARGIN
CANVAS_WID = CANVAS_HGT
TARGET_RADIUS = 4
TARGET_COLOR = 'blue'
BOID_RADIUS = 2
BOID_COLOR = 'red'

TICK_MS = 50

class App:
    # Create and manage the tkinter interface.
    def __init__(self):
        self.running = False

        # Make the main interface.
        self.window = tk.Tk()
        self.window.title('gravity_boids')
        self.window.protocol('WM_DELETE_WINDOW', self.kill_callback)
        self.window.geometry(f'{WINDOW_WID}x{WINDOW_HGT}')

        # Build the rest of the UI.
        self.build_ui()

        # We have no boids yet.
        self.boids = []

        # Make an initial target.
        self.target = Boid(self.canvas, TARGET_RADIUS, TARGET_COLOR, 1000,
            Point2d(CANVAS_WID / 2, CANVAS_HGT / 2), Vector2d(0, 0))

        # Display the window.
        self.window.focus_force()
        self.window.mainloop()

    def build_ui(self):
        # Make the drawing canvas.
        self.canvas = tk.Canvas(self.window, bg='white',
            borderwidth=1, highlightthickness=0, width=CANVAS_WID, height=CANVAS_HGT)
        self.canvas.pack(side=tk.LEFT, padx=MARGIN, pady=MARGIN, anchor=tk.NW)
        self.canvas.bind('<Motion>', self.track_mouse)

        # Make a frame to hold labels, text boxes, and buttons.
        right_frame = tk.Frame(self.window)
        right_frame.pack(side=tk.LEFT, padx=MARGIN, pady=MARGIN, anchor=tk.NW)

        # Boid mass, target mass, attraction weight, repulsion weight, target weight.
        LABEL_WID = 20
        self.boid_mass_entry = make_field(right_frame, LABEL_WID, 'Boid Mass:', 5, 100)
        self.target_mass_entry = make_field(right_frame, LABEL_WID, 'Target Mass:', 5, 1000)
        self.attraction_wgt_entry = make_field(right_frame, LABEL_WID, 'Attraction Weight:', 5, 20)
        self.repulsion_wgt_entry = make_field(right_frame, LABEL_WID, 'Repulsion Weight:', 5, 200)
        self.target_wgt_entry = make_field(right_frame, LABEL_WID, 'Target Weight:', 5, 100)
        self.neighborhood_distance_entry = make_field(right_frame, LABEL_WID, 'Neighborhood Distance:', 5, 10)

        # Make a vertical gap.
        gap = tk.Frame(right_frame, width=1, height=10)
        gap.pack(side=tk.TOP)

        # Max speed, neighbor distance, and # boids.
        self.max_speed_entry = make_field(right_frame, LABEL_WID, 'Max Speed:', 5, 100)
        self.num_boids_entry = make_field(right_frame, LABEL_WID, '# Boids:', 5, 20)

        # Make a vertical gap.
        gap = tk.Frame(right_frame, width=1, height=10)
        gap.pack(side=tk.TOP)

        # Run button.
        self.run_button = tk.Button(right_frame, text='Run', width=7, command=self.run)
        self.run_button.pack(side=tk.TOP)

    # Move the target.
    def track_mouse(self, event):
        self.target.location = Point2d(event.x, event.y)
        self.target.move_circle()

    def run(self):
        if self.running:
            # Stop running.
            self.running = False
            self.run_button.config(text='Run')
        else:
            # Start running.
            self.running = True
            self.run_button.config(text='Stop')

            # Get parameters.
            boid_mass = get_int(self.boid_mass_entry)
            self.target.mass = get_int(self.target_mass_entry)
            self.max_speed = get_int(self.max_speed_entry)
            self.attraction_wgt = get_int(self.attraction_wgt_entry)
            self.repulation_wgt = get_int(self.repulsion_wgt_entry)
            self.target_wgt = get_int(self.target_wgt_entry)
            self.neighborhood_distance = get_int(self.neighborhood_distance_entry)
            num_boids = get_int(self.num_boids_entry)

            # Make some random boids.
            self.reset_boids()
            self.boids = []
            for i in range(num_boids):
                location = self.target.location + Vector2d(
                    random.randint(-20, 20),
                    random.randint(-20, 20)
                )
                self.boids.append(Boid(self.canvas, BOID_RADIUS, BOID_COLOR,
                    boid_mass, location, Vector2d(0, 0)))

            # Go!
            self.last_time = timer()
            self.window.after(TICK_MS, self.tick)

    # Update and move the boids.
    def tick(self):
        # Get the elapsed time in seconds.
        now = timer()
        elapsed = now - self.last_time
        self.last_time = now

        # Update the boids.
        for boid in self.boids:
            boid.update(self.boids, elapsed, self.target, self.max_speed,
                self.attraction_wgt, self.repulation_wgt, self.target_wgt,
                self.neighborhood_distance)

        # Move the boids.
        for boid in self.boids:
            boid.move(elapsed)

        # If we're still running, schedule another tick.
        if self.running:
            self.window.after(TICK_MS, self.tick)

    # Destroy any existing boids and their circles.
    def reset_boids(self):
        for boid in self.boids:
            self.canvas.delete(boid.circle)
        self.boids = []

    def kill_callback(self):
        self.window.destroy()

In [50]:
App()

6666.666666666667
3446.227583977888
6097.107608496924
9090.90909090909
5352.877275721892
4331.480818242099
8574.929257125441
2936.101097573517
3987.2611141444995
5792.844463634922
13867.504905630729
3596.7500063280318
5792.844463634922
5050.762722761054
4850.7125007266595
3562.3524993954825
6984.3029576957815
6726.727939963124
4672.693135159978
4672.693135159978
8001.120919823402
5179.7441101670665
11960.74252689322
11656.746406191689
7006.78932051934
5317.939407895556
18103.247218546254
4079.3090233734792
6633.629315871878
7909.880509721935
26101.253712738635
4917.503563879528
11826.897659944496
7620.3843322902
9458.890179352271
5429.535527201058
15610.594657039179
6966.818588557739
6961.086971078955
6961.086971078955
9545.07343265559
12547.47924220426
96319.35931174805
14944.541509485025
9379.088231643163
6748.830596460044
36733.85264118905
7322.835776916913
28348.836177626734
11851.75129067585
280530.46577408793
8065.017460374987
53699.75073176774
14795.139269997528
95795.6305881365